In [1]:
!pip install langchain
!pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu
!pip install langchain-huggingface

In [2]:
import faiss
import numpy as np

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFacePipeline
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [3]:
hf_pipeline = pipeline(task="text2text-generation", model="google/flan-t5-large")
llm = HuggingFacePipeline(pipeline=hf_pipeline)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
# Read to verify it is ok
txt_path = "week3_langchain_intro/Modelos de desarrollo de software.txt"
with open(txt_path, "r", encoding="utf-8") as f:
    contenido = f.read()
contenido[:500]

'Modelos de desarrollo de software\n\n\n\nLa metodología de desarrollo de software es un marco de trabajo usado para estructurar, planificar, controlar el proceso de desarrollo de sistemas de información. \n\nA lo largo del tiempo se han desarrollado varias metodologías, cada una con sus fortalezas y sus debilidades.\n\nSu objetivo es, por un lado, determinar el orden de las fases y los procesos involucrados en el desarrollo de software y establecer los criterios de transición para pasar de una fase a la'

In [39]:
# 1. We use a txt file, so it is a Text-structured based and we need to divide this text in chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
chunks = text_splitter.split_text(text=contenido)

In [40]:
# 2. Convert to Documents list
from langchain.docstore.document import Document
docs = [Document(page_content=chunk, metadata={"source": "RAG_doc"}) for chunk in chunks]

In [41]:
# 3. Generate embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode([doc.page_content for doc in docs], show_profress_bar=True)

In [42]:
# 4. Save in FAISS
dim = embeddings.shape[1]
print(f"Embedding size: {embeddings.shape}, Dimension: {dim}")
index = faiss.IndexFlatL2(dim) # L2 = euclidian distance (menor distancia = más cercano / más parecido semánticamente)
index.add(np.array(embeddings, dtype="float32"))
print(f"Vectors quantity: {index.ntotal}")

Embedding size: (41, 384), Dimension: 384
Vectors quantity: 41


## Find the most relevant chunk

In [52]:
def find_best_passages(query: str, index, top_k: int = 1):
  query_embedding = model.encode([query])
  distances, records_index = index.search(np.array(query_embedding, dtype="float32"), k=top_k)
  top_chunks = [chunks[i] for i in records_index[0]]
  return top_chunks

## Generate answer with LLM

In [49]:
def generate_answer_rag(query: str, best_passages: list, max_length: int = 256):
    context = "\n".join(best_passages)
    prompt = f"""
            Eres un asistente experto y conversacional. Usa el texto de referencia para dar tu respuesta,
            pero agrega explicaciones adicionales, ejemplos, y tu interpretación razonable para que sea más comprensible y natural.
            Pregunta: {query}
            Contexto: {context}
            Respuesta:
            """
    output = hf_pipeline(prompt, max_new_tokens=max_length)
    return output[0]['generated_text'].split("Respuesta:")[-1].strip()

In [54]:
query = "¿Qué es el Product Owner?"
best_passages = find_best_passages(query, index, top_k=7)
answer = generate_answer_rag(query, best_passages)
print(answer)

El Product Owner o Producto es conocido como la voz del cliente. Sus responsabilidades son básicamente definir las funcionalidades del producto y priorizarlas, y aceptar, rechazar o pedir ajustes de los resultados de cada entrega. Es el documento que engloba todos los requerimientos del proyecto y se mantiene hasta su finalización, salvo que el Product Owner establezca modificaciones a los requerimientos, en cuyo caso deberá reformularse. El Product Backlog El Product Backlog es una lista que recoge todo lo que necesita el producto para satisfacer las necesidades del cliente, de la que el Product Owner es el nico responsable de su contenido y prioriza
